<a href="https://colab.research.google.com/github/01star01ek/01star01ek/blob/main/%EC%B0%90%EC%9C%BC%EB%A1%9C%20%EC%9E%98%ED%95%98%EB%8A%94%20%EC%95%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install insightface onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 125.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 120.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.6 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp311-cp311-linux_x86_64.whl size=1060438 sha256=f616035843089a8e68d930ae39e6db694ed463929ee69e7784c1558e6e910ca0
  Stored in directory: /root/.cache/pip/wheels/27/d8/22/f52d858d16cd06e7b2e6aad34a1777dcfaf000be833bbf8146
Successfully built insightface


In [5]:
import requests
import os

def download_inswapper():
    url = "https://civitai.com/api/download/models/85159"

    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open('inswapper_128.onnx', 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print("모델 다운로드 완료!")
        return True
    else:
        print(f"다운로드 실패: {response.status_code}")
        return False

download_inswapper()

모델 다운로드 완료!


True

In [14]:
import cv2
import insightface
from insightface.app import FaceAnalysis
import os

# 설정
app = FaceAnalysis()
app.prepare(ctx_id=0, det_size=(320, 320))
swapper = insightface.model_zoo.get_model('inswapper_128.onnx')

# 소스 이미지
source_img = cv2.imread('source.png')
source_faces = app.get(source_img)
source_face = source_faces[0]

# 비디오 처리
cap = cv2.VideoCapture('target.mp4')
os.makedirs('output', exist_ok=True)

frame_count = 0
saved_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 10프레임마다 처리
    if frame_count % 10 == 0:
        target_faces = app.get(frame)

        if len(target_faces) > 0:
            # 얼굴 합성
            result = swapper.get(frame, target_faces[0], source_face, paste_back=True)

            # 이미지 저장
            cv2.imwrite(f'output/frame_{saved_count:04d}.png', result)
            saved_count += 1
            print(f"저장: frame_{saved_count:04d}.png")

    frame_count += 1

cap.release()
print(f"완료! {saved_count}개 이미지 저장")

/usr/local/lib/python3.11/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:121: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (320, 320)
Applied prov

In [18]:
from glob import glob
print("영상 생성 중...")

# 이미지 파일들 가져오기
image_files = sorted(glob('output/frame_*.png'))
if len(image_files) > 0:
    # 첫 번째 이미지로 크기 확인
    first_img = cv2.imread(image_files[0])
    height, width, _ = first_img.shape

    # 비디오 라이터 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('final_result.mp4', fourcc, 30, (width, height))

    # 모든 이미지를 영상에 추가
    for img_path in image_files:
        img = cv2.imread(img_path)
        out.write(img)

    out.release()
    print(f"✅ 영상 완성! final_result.mp4 ({len(image_files)}개 프레임)")
else:
    print("❌ 저장된 이미지가 없습니다!")

영상 생성 중...
✅ 영상 완성! final_result.mp4 (36개 프레임)
